In [166]:
from modsim import *
import math
import pandas as pd
from random import randint 

# LIMESHARE 

- limeshare is...


**Below is our state with attributes:**
- number_of_scooters: 
- av_batt_life:
- size_of_region:
- send_truck:

In [184]:
limeshare = State(number_of_scooters = 20, av_batt_life = 100, size_of_region = 200, send_truck = -1, delta_by_hour = [])
limeshare

,values
number_of_scooters,20
av_batt_life,100
size_of_region,200
send_truck,-1
delta_by_hour,[]


In [168]:
def step(state, hour):
#     find number of users per hour (rush hour = more users)
#     returns a num between (0, state.number_of_scooters)
    demand = calcDemand(state.number_of_scooters, hour)
    
#     find the average ride length per rider from demand (bigger region = higher av length of use)
#     returns a num between ~(10-60)
    length_of_use = calcAverageRide(state.size_of_region, demand)
    
#     calc change in battery 
    calcBattDelta(state, demand, length_of_use)
    
#     find hour to send truck
    if(state.av_batt_life < 20 and state.send_truck == -1):
        sendTruck(state, hour)


## Calculate demand for scooters based on: 
- traffic times 
    - HIGH : going to work (8-10pm), lunch time (12-2pm), going home (4-6pm)
    - NUETRAL : early birds (6-8am), night outs (8pm-11pm)
    - LOW : all remaining times
- scooters in the area (this is a hard max) 
- randomize using round (say bad weather hour)


**Returns num of scooters with range (0, number_of_scooters)**
   - the lowest possible demand is zero and the highest is number of scooters int that region

In [169]:
def calcDemand(scooters, hour):
    if hour >= 8 and hour < 10 or hour >= 12 and hour < 14 or hour >= 16 and hour < 18:
        return randint(int(scooters/hour) + int(hour/2), scooters)
    elif hour == 6 or hour == 7 or hour >= 20 and hour < 23:  
        return randint(scooters%int(hour/2), scooters)
    else: 
        if hour == 0:
            return randint(hour, int(scooters/4))
        else:
            return randint(int(hour/6), int(scooters/4))

**The following code is used to test ideal hour ranges for our function above with demand.**
- the goal was to have a higher min for higher traffic times compatible with sccoter amounts range(20-100)


| trafffic | min  | max  | av |  
|:---|---|---|---|
| high  (13)| 13  |100   |   57.03|   
| nuetral  (20)| 1  | 100  |  48.34 |   
| low (4)|  0 |  25 |  12.55 |   
[This is for 100 scooters]


| trafffic | min  | max  | av |  
|:---|---|---|---|
| high  (13)|   7|20   |   13.08|   
| nuetral  (20)| 0  | 20  |  10.6 |   
| low (4)|  0 | 5  | 2.37  |   
[This is for 20 scooters]

In [170]:
# have to rewrite minimum var from min to 'mini' below bc min is reserved 
hours, mini, max, av = 4, 100, 0, 0
for i in range(1, 100):
    store = calcDemand(100, hours)
    if store < mini: mini = store
    if store > max: max = store
    av += store
print(mini, max, av/100)

0 25 12.39


## Caluculate average ride length of scooters used 

**factors the affect the ride length**
- area of region (the higher the area the longer the ride)
- demand (if there is more demand, chances are the rides are longer)

**Ranges for inputs**
- demand: from 10-70 ore more
- areas: 200 - 2000 acres

**returns average length scooters used for the hour with range (~10, ~60) min**
- check the test methods below 

In [171]:
def calcRide(area):
    return randint(int(area/60), int(area/60) + area%60)

def calcAverageRide(area, demand):
    av = 0
    for i in range (0, demand): 
        av += calcRide (area);
    return av/demand if demand > 0 else 0

Test the method above with av demand of 30 scooters and area range 500-2k acres

In [172]:
print(calcAverageRide(500, 30), calcAverageRide(1000, 30), calcAverageRide(2000, 30))

18.466666666666665 35.666666666666664 43.63333333333333


## Calc changes in average battery life of region from demand 
We came up with a simple formula in class. The whole point of the fucntion is to check how much battery goes down from sccoter usage 

### CASE STUDY:
- lets say before method is called we have 20 scooters (s) in region with av battery life of 100 percent (c) 
- this means all the scooters in this regions is fully charged
- if let's say the demand is only 1 scooter (t) and we say that scooter charge after use goes down 10 percent (d)  
- then the average for all the scooters decreases by a little but exactly how much ?  
<br/>

#### The new average battery life of the region can be calculated by finding... 
- **batt_not_used:** The av of scooters that were'nt used times the charge 
    - so 19\*100 OR (s-t)\*c = 1900
- **batt_used:** the average of scooters that were used times the charge delta 
    - so 1\*90 OR t(c-d) = 90
- the new average of the region is thus 
    - (batt_not_used + batt_used) / s = (1900 + 90) / 20 = 99.5

THE FORMULA
- ((s-t)*c + t*(c- d)) / s
    

**Inputs range**
- state: as defined
- demand: same as inputs range for calcDemand() which is (0, state.number_of_scooters) or (0-120ish) for this model
- av_length_of_use: same as inputs range for calcAvRide() which is (~10, ~60)

In [181]:
def calcBattDelta(state, demand, av_length_of_use):
    s = state.number_of_scooters 
    c = state.av_batt_life
    t = demand
    d = (av_length_of_use / 60)*100
    
    state.av_batt_life =  math.floor(((s-t)*c + t*(c- d)) / s)
    state.delta_by_hour.append(state.av_batt_life)
        

def sendTruck(state, hour):
    state.send_truck = hour;

In [182]:
def run_simulation(state, num_steps):     
    results = TimeSeries()
    for hour in range(num_steps):
        #  this is our line in the sand to prevent calculating after our desired purpose for the model has been found
        if(state.send_truck != -1):
            return
        step(state, hour)
        results[hour] = state.av_batt_life

In [183]:
limeshare = State(number_of_scooters = 20, av_batt_life = 100, size_of_region = 200, send_truck = -1, delta_by_hour = [])
run_simulation(limeshare, 24)
limeshare

,values
number_of_scooters,20
av_batt_life,15
size_of_region,200
send_truck,9
delta_by_hour,"[94, 94, 91, 91, 91, 86, 65, 62, 39, 15]"


In [176]:
def runModel():
#     tuple array ( size_of_region, send_truck) with fixed number_of_scooters at 20
    regions = linspace(200, 2000, 10)
    results = []
    for i in regions:
        limeshare = State(number_of_scooters = 20, av_batt_life = 100, size_of_region = round(i + 1), send_truck = -1, delta_by_hour = [])
        run_simulation(limeshare, 24)
        data = (limeshare.size_of_region, limeshare.send_truck)
        results.append(data)
    return results 

In [177]:
sample_regions = runModel()
print(sample_regions)

TypeError: run_simulation() missing 1 required positional argument: 'delta_by_hour'

In [ ]:
def runModel():
#     tuple array (number_of_scooters, send_truck) with fixed size_of_region at 2
    scooters = linspace(20, 70, 10)
    results = []
    for i in scooters:
        limeshare = State(number_of_scooters = int(i), av_batt_life = 100, size_of_region = 1000, send_truck = -1, delta_by_hour = [])
        run_simulation(limeshare, 24)
        data = (limeshare.number_of_scooters, limeshare.send_truck)
        results.append(data)
    return results 

In [ ]:
sample_scooters = runModel()
print(sample_scooters)

In [ ]:
region_data = linspace(200, 2000, 10)


In [ ]:
scooter_data = linspace(20, 80, 10)

In [ ]:
truck_data = [[0 for x in region_data] for y in scooter_data] 

In [ ]:
def runModel():
    for row, reigon in enumerate(region_data):
        for col, scooters in enumerate(scooter_data):
            limeshare = State(number_of_scooters = round(scooters), av_batt_life = 100, size_of_region = round(reigon), send_truck = -1, delta_by_hour = [])
            run_simulation(limeshare, 24)
            truck_data[row][col] = limeshare.send_truck

runModel()
truck_data

In [ ]:
# My reaction when the above code printed "HOLY SHIT!"

In [ ]:
new_dataFrame = pd.DataFrame(data=truck_data, index=round(region_data, 2), columns=round(scooter_data, 2))
new_dataFrame